<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/RNN_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def download_and_read(url):
  local_file = url.split('/')[-1]
  local_file = local_file.replace("%20", " ")
  p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir=".")
  local_folder = os.path.join("drive/MyDrive/Datasets/UCI_ML_ES", local_file.split('.')[0])
  labeled_sentences = []
  for labeled_filename in os.listdir(local_folder):
    if labeled_filename.endswith("_labelled.txt"):
      with open(os.path.join(local_folder, labeled_filename), "r") as f:
        for line in f:
          sentence, label = line.strip().split("\t")
          labeled_sentences.append((sentence, label))

  return labeled_sentences
labeled_sentences = download_and_read(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
)
sentences = [s for (s, l) in labeled_sentences]
labels = [l for (s, l) in labeled_sentences]

90112/84188 [================================] - 0s 1us/step


In [3]:
#문장의 각 단어를  token으로 변환하기
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size : {:d}".format(vocab_size))

word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size : 5271


In [4]:
#정적인 입력크기를 정하기 위해, 대부분 문장을 수용할 수 있을 만큼의 시퀀스 길이를 정한다.
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])
#75% : 16단어 , 80% 18단어, 99% 36단어, 최대길이 71단어.

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [5]:
#최대 단어길이 64로 설정
max_seqlen = 64

#데이터 셋 생성
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
#남은 부분은 0으로 채워준다.
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels, dtype=np.float32)
dataset = tf.data.Dataset.from_tensor_slices((sentences_as_ints, labels_as_ints))

dataset = dataset.shuffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [6]:
print(train_dataset)

<BatchDataset shapes: ((None, 64), (None,)), types: (tf.int32, tf.float32)>


In [7]:
class SentimentAnalysisModel(tf.keras.Model):
  def __init__(self, vocab_size, max_seqlen, **kwargs):
    super(SentimentAnalysisModel, self).__init__(**kwargs)
    self.embedding = tf.keras.layers.Embedding(vocab_size, max_seqlen)
    self.bilstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(max_seqlen)
    )
    self.dense = tf.keras.layers.Dense(64, activation='relu')
    self.out = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, x):
    x = self.embedding(x)
    x = self.bilstm(x)
    x = self.dense(x)
    x = self.out(x)
    return x

model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
model.build(input_shape=(batch_size, max_seqlen))
model.compile(
    loss="binary_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
)
model.summary()

Model: "sentiment_analysis_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  337408    
_________________________________________________________________
bidirectional (Bidirectional multiple                  66048     
_________________________________________________________________
dense (Dense)                multiple                  8256      
_________________________________________________________________
dense_1 (Dense)              multiple                  65        
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [8]:
data_dir = "drive/MyDrive/Colab Notebooks/models/RNN/sentiment_analysis/"
logs_dir = "drive/MyDrive/Colab Notebooks/logs/RNN/sentiment_analysis/"
best_model_file = os.path.join(data_dir, "best_model.h5")

In [11]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file, save_weights_only=True, save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, 
    validation_data=val_dataset,
    callbacks=[checkpoint, tensorboard])


Epoch 1/10
29/29 [==============================] - 8s 153ms/step - loss: 0.6918 - accuracy: 0.5128 - val_loss: 0.6799 - val_accuracy: 0.5600
Epoch 2/10
29/29 [==============================] - 3s 103ms/step - loss: 0.6180 - accuracy: 0.7133 - val_loss: 0.4925 - val_accuracy: 0.8500
Epoch 3/10
29/29 [==============================] - 3s 100ms/step - loss: 0.4020 - accuracy: 0.8611 - val_loss: 0.2471 - val_accuracy: 0.9150
Epoch 4/10
29/29 [==============================] - 3s 99ms/step - loss: 0.2280 - accuracy: 0.9267 - val_loss: 0.1728 - val_accuracy: 0.9450
Epoch 5/10
29/29 [==============================] - 3s 98ms/step - loss: 0.1526 - accuracy: 0.9500 - val_loss: 0.2388 - val_accuracy: 0.9100
Epoch 6/10
29/29 [==============================] - 3s 101ms/step - loss: 0.1246 - accuracy: 0.9633 - val_loss: 0.0738 - val_accuracy: 0.9800
Epoch 7/10
29/29 [==============================] - 3s 102ms/step - loss: 0.0982 - accuracy: 0.9672 - val_loss: 0.0901 - val_accuracy: 0.9750
Epoch 8/

In [12]:
best_model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
best_model.build(input_shape=(batch_size, max_seqlen))
best_model.load_weights(best_model_file)
best_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss : {:.3f}, test accuracy : {:.3f}".format(test_loss, test_acc))

16/16 [==============================] - 2s 28ms/step - loss: 0.0565 - accuracy: 0.9890
test loss : 0.057, test accuracy : 0.989
